# 1 - Overview/Introduction:

### In this tutorial we will learn 3 key things to using Architector:

**(A)** How to define basic input dictionaries for Architector.

**(B)** How to understand and visualize outputs from Architector.

**(C)** How to play with potentially important parameters for getting at chemical meaning.

## Starting from **(A)**: 
Architector operates entirely within python for the general user taking an input dictionary, returning an output dictionary.

In [ ]:
# Initialize Input Dictionary:
inputDict = dict()

For the first example we will make and Iron Hexa Aqua Complex : [Fe(H<sub>2</sub>O)<sub>6</sub>]<sup>2+</sup> / [Fe(H<sub>2</sub>O)<sub>6</sub>]<sup>3+</sup>

There are 3 high-level inputs to Architector in an input dictionary to be aware of:

## 1. Core: Dictionary:

Indicates what metal is present, what its coordination number (CN) or number of of connections are, or specific core coordination. So for Iron Hexa-Aqua we will need to specify two parameters:

In [ ]:
# Initialize Core Dictionary
coreDict = dict()

# Specify the metal:
coreDict['metal'] = 'Fe'

# Specify the coordination number (CN):
coreDict['coreCN'] = 6

### That's it! Now we can add core to the input dictionary:

In [ ]:
# Add the core to the input dictionary:
inputDict['core'] = coreDict

## 2. Ligands Dictionary:

The ligands input to Architector specifies which ligands and the relative numbers of ligands passed. 

The ligands section of the inputDict is at the base level a python list of ligands represented as a dictionary:

In [ ]:
# Initilize Ligand List
ligList = []

# Define Water dictionary
water = dict()

The simplest way to define ligands is from only the ligands SMILES string and the list of coordinating atoms.

[SMILES](https://en.wikipedia.org/wiki/Simplified_molecular-input_line-entry_system) is one of the most common methods for representing molecules for chemists. For water (H<sub>2</sub>O), and most common chemicals, we can find their representative smiles strings even on [wikipedia](https://en.wikipedia.org/wiki/Properties_of_water) or on [pubchem](https://pubchem.ncbi.nlm.nih.gov/compound/Water) from a quick google search.

For water, the SMILES is 'O'. 

In [ ]:
# Add SMILES definition
water['smiles'] = 'O'

Since there is only one heavy element (Z > 1) in water, the only coordination site for the molecule is the first element, which corresponds to a (0-indexed) site of [0]. For more on SMILES and coordination site identification see the second tutorial: 2-Ligand_Identification.ipynb.

In [ ]:
# Add coordination site list
water['coordList'] = [0]

### Finally, we need to add 6 waters to the input dictionary for Iron Hexa-Aqua:

In [ ]:
# Add six copies of water to the ligand list:
ligList += [water] * 6 

# Add the ligands to the inputDictionary
inputDict['ligands'] = ligList

## 3. Parameters

The parameters section of architector can give you broad leeway to specify how you want the construction of the molecules to occur.

A basic parameters input requires just an empty dictionary. To increase this complexity by just a bit we will add the desired oxidation state to the parameters.  (giving: [Fe(H<sub>2</sub>O)<sub>6</sub>]<sup>2+</sup>)

In [ ]:
# Initialize Parameters dictionary
parameters = dict()

# Specify oxidation state of 2 for the metal
parameters['metal_ox'] = 2

### And add these parameters to the input dictionary:

In [ ]:
inputDict['parameters'] = parameters

### Now we have a fully assembled input dictionary for [Fe(H<sub>2</sub>O)<sub>6</sub>]<sup>2+</sup>

This is quite easy to print out in a jupyter notebook:

In [ ]:
inputDict

#### Note that a FULL description of potential input dictionary parameters and values can be found in the README.md file.

In [ ]:
# Now we Import the main building functionalities and the in-built visualization of architector:
from architector import build_complex, view_structures

#### Now, we build the complex!

Building the first complex should take just a couple seconds:

In [ ]:
# We can see it print out information as it is processing.
# The last line should be: 'ComplexSanity:  True'
out = build_complex(inputDict)

## Now, onto **(B)** :  Understanding and visualizing output from architector

There's obviously a lot going on here:

In [ ]:
out

#### So let's first just visualize the structures:

In [ ]:
view_structures(out)

In [ ]:
# We can also add labels to describe the structures:
labels = list(out.keys()) # Here, I am just pulling out the keys describing each structure
view_structures(out,labels=labels)

#### We can also look at what architector assigns as the defualt charge and spin for verification

In [ ]:
key = labels[0] # Pull out the first structure to get spin/charge states
print('Metal Oxidation State: ',out[key]['metal_ox'])
print('Total System Charge: ',out[key]['total_charge'])
print('Total N Unpaired Electrons (spin): ',out[key]['calc_n_unpaired_electrons'])

#### Note that the default spin state for Fe in Architector is high-spin (4 unpaired electrons means spin multiplicity = 5)!

## Now for (C), we want to generate  [Fe(H<sub>2</sub>O)<sub>6</sub>]<sup>3+</sup> to see if there's a difference!

Looking at different charge and spin states can be key to understanding the structure and function of different first-row transition metal complexes as in [This Work](https://pubs.rsc.org/en/content/articlelanding/2020/cp/d0cp02977g).

We can copy the inputDict and simply modify in place to create Fe3+ Hexa-Aqua!

In [ ]:
# Import copy
import copy

In [ ]:
# Copy inputDict
new_inputDict = copy.deepcopy(inputDict)
# Set the metal oxidation state to 3 instead
new_inputDict['parameters']['metal_ox'] = 3

Printing out the new input dictionary reveals the slight shift:

Note that during building the is_actinide, and original metal flags get added in case an actinide is passed as an input.

In [ ]:
new_inputDict

### Now we can build the [Fe(H<sub>2</sub>O)<sub>6</sub>]<sup>3+</sup> 

In [ ]:
# Build new molecule (Takes maybe 30 seconds)
out1 = build_complex(new_inputDict)

In [ ]:
# Let's visualize the strucutres again:
view_structures(out1)

### How about the spin and charge?

The structures look very similar, so let's check the spin and charge:

In [ ]:
key = list(out1.keys())[0]
print('Metal Oxidation State: ',out1[key]['metal_ox'])
print('Total System Charge: ',out1[key]['total_charge'])
print('Total N Unpaired Electrons (spin): ',out1[key]['calc_n_unpaired_electrons'])

So we can see we've created both High Spin (HS) [Fe(H<sub>2</sub>O)<sub>6</sub>]<sup>3+</sup> in the out1 dictionary, and [Fe(H<sub>2</sub>O)<sub>6</sub>]<sup>2+</sup> in the out dictionary.

For simplicity and tracking sake, let's rename these variables accordingly:

In [ ]:
hs_fe2_dict = out
hs_fe3_dict = out1

## Still in (C), let's measure the difference in bond distances betwee the two charge state structures!

We will be using numpy package for analysis:

In [ ]:
import numpy as np

#### Let's look at the High-Spin (HS) Fe-O distances for both the octahedral 2+ and 3+ forms using the following function:

For this function we will also be using several of the built-in functionality of the [ASE Atoms](https://wiki.fysik.dtu.dk/ase/ase/atoms.html) output included in every output dictionary.

In [ ]:
def avg_fe_o_dist(ase_atoms):
    # Have ase atoms gives all the functionality of ase!
    symbols = np.array(ase_atoms.get_chemical_symbols()) # List of chemical symbols
    distances = ase_atoms.get_all_distances() # Matrix (Natoms x Natoms) of distances in Angstroms
    # Pull out the indices of Fe, and O:
    fe_ind = np.where(symbols == 'Fe')[0]
    o_inds = np.where(symbols == 'O')[0]
    # Now tablulate Fe-O distances
    dists = [distances[fe_ind,x] for x in o_inds]
    avg_dists = np.mean(dists) # And take the average
    return avg_dists

We first look at Fe2+-O distances.

In [ ]:
fe2key = list(hs_fe2_dict.keys())[0]
hs_fe2_avg_dist = avg_fe_o_dist(hs_fe2_dict[fe2key]['ase_atoms'])
print('Average Fe2+-O distance (Angstroms):', hs_fe2_avg_dist)

In [ ]:
# Same procedure for F3+-O!
fe3key = list(hs_fe3_dict.keys())[0]
hs_fe3_avg_dist = avg_fe_o_dist(hs_fe3_dict[fe3key]['ase_atoms'])
print('Average F3+-O distance (Angstroms):', hs_fe3_avg_dist)

### So we can see that the 3+ Fe-O distance is slightly shorter than the 2+ Fe-O distance.

This is in agreement with intuition, where more highly charged metal center attracts negatively charged O stronger!

## What about if we want to look at different spin states, e.g. Low-Spin (LS) Configurations?

Now we can simply re-copy the input dictionary and edit again to examine low-spin configurations:

In [ ]:
fe2_ls_inputDict = copy.deepcopy(inputDict)

In [ ]:
# Here, we assign the metal spin to be 0 and let architector ultimately assign the spin:
fe2_ls_inputDict['parameters']['metal_ox'] = 2
fe2_ls_inputDict['parameters']['metal_spin'] = 0

Quick spot-check to make sure the parameters are what we'd like for LS Fe2+ Hexa-Aqua

In [ ]:
fe2_ls_inputDict

Looks good! Now we can build the LS Fe2+ complex:

In [ ]:
ls_fe2_dict = build_complex(fe2_ls_inputDict)

And again we can visualize!

In [ ]:
view_structures(ls_fe2_dict)

### Wait! In some cases for LS - only one unique geometry is generated!

This is because Architector automatically removes duplicate geometries unless otherwise requested.

For verification, let's check the spin/charge states of the complex:

In [ ]:
key = list(ls_fe2_dict.keys())[0]
print('Metal Oxidation State: ',ls_fe2_dict[key]['metal_ox'])
print('Total System Charge: ',ls_fe2_dict[key]['total_charge'])
print('Total N Unpaired Electrons (spin): ',ls_fe2_dict[key]['calc_n_unpaired_electrons'])

# LS Fe2+ Looks good - how about LS Fe3+?

Here, we again copy the input dictionary:

In [ ]:
# How about for LS Fe3+ ?
fe3_ls_inputDict = copy.deepcopy(new_inputDict)

To highlight that Architector automatically determines the closest chemically-relevant spin, we will again assign the spin of LS Fe3+ to 0, when we know there is at least 1 unpaired electron (meaning LS should be 1!)

In [ ]:
fe3_ls_inputDict['parameters']['metal_spin'] = 0
fe3_ls_inputDict # Print out the assemble dictionary

Now build the Fe3+ LS structure:

In [ ]:
ls_fe3_dict = build_complex(fe3_ls_inputDict)

In [ ]:
view_structures(ls_fe3_dict)

Note that we might be back to all three structures for LS Fe3+!

Here, we will again check that spin and charge have been correctly assigned:

In [ ]:
key = list(ls_fe3_dict.keys())[0]
print('Metal Oxidation State: ',ls_fe3_dict[key]['metal_ox'])
print('Total System Charge: ',ls_fe3_dict[key]['total_charge'])
print('Total N Unpaired Electrons (spin): ',ls_fe3_dict[key]['calc_n_unpaired_electrons'])

### Looks like low-spin Fe3+,  and that the 0 was automaticlaly shifted to 1 upaired electron for the Fe3+ system.

If you try and assign an un-physical spin state Architector will automatically correct it to the closest possible physically-meaningful spin state!

### Let's also examine the Fe-O bond distances in the LS configurations.

Here we'd expect LS Fe-O bond lengths to be shorter than their HS counterparts!

In [ ]:
ls_fe2_key = list(ls_fe2_dict.keys())[0]
ls_fe2_atoms = ls_fe2_dict[ls_fe2_key]['ase_atoms']
ls_fe2_avg_dist = avg_fe_o_dist(ls_fe2_dict[ls_fe2_key]['ase_atoms'])
print('Average LS Fe2+-O distance (Angstroms): ',ls_fe2_avg_dist)

In [ ]:
ls_fe3_key = list(ls_fe3_dict.keys())[0]
ls_fe3_atoms = ls_fe3_dict[ls_fe3_key]['ase_atoms']
ls_fe3_avg_dist = avg_fe_o_dist(ls_fe3_dict[ls_fe3_key]['ase_atoms'])
print('Average LS Fe3+-O distance (Angstroms): ',ls_fe3_avg_dist)

### We see the same trend for Fe2+-O vs. Fe3+-O distances in the LS vs. HS! 

How about if we plot out all of these Fe-O distances for different oxidation states:

In [ ]:
# Here, we'll use matplotib to generate a plot!
import matplotlib.pyplot as plt

Basic plotting with metal oxidation state and average distances:

In [ ]:
x = [2,3] # Oxidation states
plt.scatter(x,[ls_fe2_avg_dist,ls_fe3_avg_dist],label='Low Spin',color='b')
plt.scatter(x,[hs_fe2_avg_dist,hs_fe3_avg_dist],label='High Spin',color='r')
plt.xlim(1.5,3.5)
plt.xticks([2,3])
plt.legend()
plt.xlabel('Metal Oxidation State')
plt.ylabel('Average Fe-O distance ($\AA$)')

## Looks exactly like what we'd expect from chemical intuition!

The [XTB](https://xtb-docs.readthedocs.io/) methods Architector use in the background capture these chemical trends near-perfect!

## Finally, for any of these structures, we can write out potential structures to use in any external electronic structure code!

Uncomment (remove the #s) and run the cell below to get a labelled .xyz file for LS Fe3+ - Hexa-Aqua!

In [ ]:
# label = list(ls_fe3_dict.keys())[0]
# ase_atoms = ls_fe3_dict[label]['ase_atoms']
# ase_atoms.write(label+'.xyz')

# Conclusion

In this tutorial we used Fe Hexa Aqua as an example to learn 3 key basic features of Architector:

**(A)** How to define basic input dictionaries for Architector.

**(B)** How to understand and visualize outputs from Architector.

**(C)** How to play with potentially important parameters for getting at chemical meaning.